In [4]:
def cleanData(csv_path):
    '''
    cleans up san diego traffic data for api processing

    Params
    csv_path: str of path to csv file
    '''
    assert isinstance(csv_path, str)
    assert csv_path[-4:] =='.csv'
    import pandas as pd
    
    df = pd.read_csv(csv_path)
    df = df[df['count'] > 5]
    df = df[df['lat_x'].notna()]
    df = df[df['lon_x'].notna()]
    df = df[df['lat_y'].notna()]
    df = df[df['lon_y'].notna()]
    df.reset_index(drop=True)
    df.to_csv('../processed_datasets/sd_cleaned.csv')

cleanData('../processed_datasets/sd_data_processed.csv')

In [9]:
def points(start_lat,start_lng,end_lat,end_lng):
    '''
    makes an api call to route and directions to get a list of coordinates of the shortest path between two geographical coordinates

    Params
    start_lat: the origin lattitude
    start_lng: the origin longitude
    end_lat: the destination lattitude
    end_lng: the destination longitude
    '''
    assert isinstance(start_lat, str)
    assert isinstance(start_lng, str)
    assert isinstance(end_lat, str)
    assert isinstance(end_lng, str)

    import requests

    url = "https://route-and-directions.p.rapidapi.com/v1/routing"
    querystring = {"waypoints":f"{start_lat},{start_lng}|{end_lat},{end_lng}","mode":"drive"}
    headers = {
        "X-RapidAPI-Key": "INSERT-YOUR-KEY-HERE",
        "X-RapidAPI-Host": "route-and-directions.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    return response


def getCoordinates(csv_file):
    '''
    reads a csv file of starting and ending locations, creates a csv file of points of all the paths traveled

    Params
    csv_file- str to csv file location
    '''
    assert isinstance(csv_file, str)

    import pandas as pd
    df_tester = pd.read_csv(csv_file)

    points_list = []
    for i in range(len(df_tester)):
        print(i*100/(len(df_tester)), '% Complete')

        count = str(df_tester['count'].loc[i])
        start_lat = str(df_tester['lat_x'].loc[i])
        start_lng = str(df_tester['lon_x'].loc[i])
        end_lat = str(df_tester['lat_y'].loc[i])
        end_lng = str(df_tester['lon_y'].loc[i])

        #makes an api call to get a list of coordinates as a JSON request response
        coordinates = points(start_lat,start_lng,end_lat,end_lng)

        if coordinates.ok:
            points_json = coordinates.json()
            points_list.append(points_json)
        else:
            points_list.append("nope")

    #adds the coordinates to the df
    df_tester['coordinates'] = points_list

    #cleans the data of error responses from the api
    df_tester = df_tester[df_tester['coordinates'] != "nope"]
    df_tester = df_tester[df_tester['coordinates'] != "{'statusCode': 400, 'error': 'Bad Request', 'message': 'No path could be found for input'}"]
    df_tester.to_csv('../processed_datasets/sd_final.csv')


getCoordinates('../processed_datasets/sd_cleaned.csv')

0.0 % Complete
0.9900990099009901 % Complete
1.9801980198019802 % Complete
2.9702970297029703 % Complete
3.9603960396039604 % Complete
4.9504950495049505 % Complete
5.9405940594059405 % Complete
6.930693069306931 % Complete
7.920792079207921 % Complete
8.910891089108912 % Complete
9.900990099009901 % Complete
10.891089108910892 % Complete
11.881188118811881 % Complete
12.871287128712872 % Complete
13.861386138613861 % Complete
14.851485148514852 % Complete
15.841584158415841 % Complete
16.831683168316832 % Complete
17.821782178217823 % Complete
18.81188118811881 % Complete
19.801980198019802 % Complete
20.792079207920793 % Complete
21.782178217821784 % Complete
22.77227722772277 % Complete
23.762376237623762 % Complete
24.752475247524753 % Complete
25.742574257425744 % Complete
26.73267326732673 % Complete
27.722772277227723 % Complete
28.712871287128714 % Complete
29.702970297029704 % Complete
30.693069306930692 % Complete
31.683168316831683 % Complete
32.67326732673267 % Complete
33.